In [1]:
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
from numpy import random
import matplotlib.image as mpimg
from scipy.optimize import curve_fit

## Formula that describes the ellipse on the Armenteros-odolanski plot.
def func(x, m_Ks, m_pi):
    return np.sqrt(abs((-m_Ks)**2*(1.-x**2)/4.-m_pi**2))

In [ ]:
data=np.loadtxt("KS.dat") #loading the actual collision data
data=data[(np.arange(data.shape[0])[data[:,1]>115.5]),:] #cutting off the Lambdas and leaving only KS

realdata=data[:,(2,1)] #extracting just the pt and alpha from the real collision data
realdata=np.array(realdata)

In [ ]:
### Run1 ###

## Generating pseudo datasets, binngin and calculating the Theil index for every one of them.

nps=1000 #number of pseudo datasets

y_binnum=100 #number of bins along the y-axis
x_binnum=100 #number of bins along the x-axis

y_binWidth=350./y_binnum #height of one bin
x_binWidth=2./x_binnum   #length of one bin

Theil=np.zeros((nps,3)) #[Theil index,Kaon mass,pion mass]

StartMatrix=np.zeros((y_binnum,x_binnum)) #Making the Armenteros-Podolanski plot of the real data.
for e in range(realdata.shape[0]): #Binning the real data
    i=realdata[e,1]//y_binWidth #finding the y-coordinate of the bin where this event belongs
    j=(realdata[e,0]+1)//x_binWidth #finding the x-coordinate of the bin where this event belongs
    StartMatrix[i,j]+=1 #adding the event to its bin
 
for p in range(nps): #going trough the pseudo datasets
    kaon=random.uniform(450.,550.)
    pion=random.uniform(100.,200.)
    events=[]
    matrix=zeros((y_binnum,x_binnum)); #Making the Armenteros-Podolanski plot and generating bins on it.
    matrix=StartMatrix.copy()
    for e in range(data.shape[0]): #going trough every event in one specific pseudo dataset
        alpha=random.uniform(-1.,1.)
        i=func(alpha,kaon,pion)//y_binWidth #finding the y-coordinate of the bin where this event belongs
        j=(alpha+1)//x_binWidth #finding the x-coordinate of the bin where this event belongs
        matrix[i,j]+=1 #adding the event to its bin
    T=0
    for i_e in range(y_binnum): #making an array of bins that have any events
        for j_e in range(x_binnum):
            if matrix[i_e,j_e]>0:
                events=np.append(events,matrix[i_e,j_e])    
    events=np.array(events)
    mean=np.mean(events)
    for i in range(events.shape[0]): #calculating the Theil index
        T1=events[i]*np.log(events[i]/mean)/mean
        T+=T1
    T/=events.shape[0]*np.log(events.shape[0])
    Theil[p]=[T,kaon,pion] 

np.savetxt('pseudo-Theilindex-run1-1000.txt', Theil, delimiter=',')     

In [ ]:
TheilGen=np.loadtxt("pseudo-Theilindex-run1-1000.txt",delimiter=',') #Loading the dots from the run.
TheilGen=TheilGen[TheilGen[:, 0].argsort()[::-1]] #Sorting the dots by the Theil index.
TheilGen=TheilGen[:100] #Extracting the 10% dots with the highest Theil index.

###
### Drawing an ellipse around the top 10% dots.
###

ellipsedata=TheilGen[:,[1,2]].T

# Step 1: center the data to get the translation vector.
print 'Translation', ellipsedata.mean(axis=1)
[mean_x,mean_y ]=ellipsedata.mean(axis=1)
ellipsedata -= numpy.reshape(ellipsedata.mean(axis=1), (2, 1))

# Step 2: perform PCA to find rotation matrix.
scatter = numpy.dot(ellipsedata, ellipsedata.T)
eigenvalues, transform = numpy.linalg.eig(scatter)
print 'Rotation matrix', transform

# Step 3: Rotate back the data and compute radii.
# You can also get the radii from smaller to bigger
# with 2 / numpy.sqrt(eigenvalues)
rotated = numpy.dot(numpy.linalg.inv(transform), ellipsedata)
print 'Radii', 2 * rotated.std(axis=1)
[a, b]= 2 * rotated.std(axis=1)

# Drawng the ellipse
ex = np.linspace(0,a,10000)
ey =np.array(  b *np.sqrt(1-(ex / a)**2 ) )
ex=np.array([np.concatenate((ex,ex,-ex,-ex))])
ey=np.array([np.concatenate((ey,-ey,ey,-ey),axis=1)])
ellipse=np.concatenate((ex,ey))
transform=np.array(transform)
ellipse=numpy.dot(transform, ellipse)
ellipse[0] += mean_x
ellipse[1] += mean_y
plot(ellipse[0],ellipse[1],'.r')

#Generating new 1000 pseudo datasets in the ellipse area for the next iteration.
n=0
Ellipse_x=[]
Ellipse_y=[]
while n<1000:
    X =np.random.uniform(-a,a)
    Y =np.random.uniform(-b,b)
    d = (X / a) ** 2 + (Y / b) ** 2
    if d<1:
        Ellipse_x=np.append(Ellipse_x,X)
        Ellipse_y=np.append(Ellipse_y,Y)
        n=n+1
Ellipse_x=np.array([Ellipse_x])
Ellipse_y=np.array([Ellipse_y])

Ellipse = np.concatenate((Ellipse_x, Ellipse_y),axis=0)

Ellipse = numpy.dot(transform, Ellipse)
Ellipse[0, :] += mean_x
Ellipse[1, :] += mean_y

In [ ]:
Theil=np.loadtxt("pseudo-Theilindex-run1-1000.txt",delimiter=',')

from matplotlib import pyplot as plt

x = Theil[:,1]
y = Theil[:,2]
z = Theil[:,0]
color = [str(item/255.) for item in z]

plt.figure(figsize=(15,10))
plt.scatter(x, y, c=z, s=40, cmap=plt.cm.Blues)
plt.xlabel('generated Kaon mass [MeV/c^2]')
plt.ylabel('generated pion mass [MeV/c^2]')
cbar=plt.colorbar()
cbar.set_label('Theil index')
plot(ellipse[0],ellipse[1],'.r') #Potting the red ellipse.

savefig("run1-bin100x100.png")


In [ ]:
### Run2 ###

## Generating pseudo datasets, binngin and calculating the Theil index for every one of them.

nps=1000 #number of pseudo datasets

y_binnum=100 #number of bins along the y-axis
x_binnum=100 #number of bins along the x-axis

y_binWidth=350./y_binnum #height of one bin
x_binWidth=2./x_binnum   #length of one bin

Theil=np.zeros((nps,3)) #[Theil index,Kaon mass,pion mass]
 
for p in range(nps): #going trough the pseudo datasets
    kaon=Ellipse[0,p]
    pion=Ellipse[1,p]
    events=[]
    matrix=zeros((y_binnum,x_binnum)); #Making the Armenteros-Podolanski plot and generating bins on it.
    matrix=StartMatrix.copy()
    for e in range(data.shape[0]): #going trough every event in one specific pseudo dataset
        alpha=random.uniform(-1.,1.)
        i=func(alpha,kaon,pion)//y_binWidth #finding the y-coordinate of the bin where this event belongs
        j=(alpha+1)//x_binWidth #finding the x-coordinate of the bin where this event belongs
        matrix[i,j]+=1 #adding the event to its bin
    T=0
    for i_e in range(y_binnum): #making an array of bins that have any events
        for j_e in range(x_binnum):
            if matrix[i_e,j_e]>0:
                events=np.append(events,matrix[i_e,j_e])    
    events=np.array(events)
    mean=np.mean(events)
    for i in range(events.shape[0]): #calculating the Theil index
        T1=events[i]*np.log(events[i]/mean)/mean
        T+=T1
    T/=events.shape[0]*np.log(events.shape[0])
    Theil[p]=[T,kaon,pion] 

np.savetxt('pseudo-Theilindex-run2-1000.txt', Theil, delimiter=',')

In [ ]:
TheilGen=np.loadtxt("pseudo-Theilindex-run2-1000.txt",delimiter=',') #Loading the dots from the run.
TheilGen=TheilGen[TheilGen[:, 0].argsort()[::-1]] #Sorting the dots by the Theil index.
TheilGen=TheilGen[:100] #Extracting the 10% dots with the highest Theil index.

###
### Drawing an ellipse around the top 10% dots.
###

ellipsedata=TheilGen[:,[1,2]].T

# Step 1: center the data to get the translation vector.
print 'Translation', ellipsedata.mean(axis=1)
[mean_x,mean_y ]=ellipsedata.mean(axis=1)
ellipsedata -= numpy.reshape(ellipsedata.mean(axis=1), (2, 1))

# Step 2: perform PCA to find rotation matrix.
scatter = numpy.dot(ellipsedata, ellipsedata.T)
eigenvalues, transform = numpy.linalg.eig(scatter)
print 'Rotation matrix', transform

# Step 3: Rotate back the data and compute radii.
# You can also get the radii from smaller to bigger
# with 2 / numpy.sqrt(eigenvalues)
rotated = numpy.dot(numpy.linalg.inv(transform), ellipsedata)
print 'Radii', 2 * rotated.std(axis=1)
[a, b]= 2 * rotated.std(axis=1)

# Drawng the ellipse
ex = np.linspace(0,a,10000)
ey =np.array(  b *np.sqrt(1-(ex / a)**2 ) )
ex=np.array([np.concatenate((ex,ex,-ex,-ex))])
ey=np.array([np.concatenate((ey,-ey,ey,-ey),axis=1)])
ellipse=np.concatenate((ex,ey))
transform=np.array(transform)
ellipse=numpy.dot(transform, ellipse)
ellipse[0] += mean_x
ellipse[1] += mean_y
plot(ellipse[0],ellipse[1],'.r')

#Generating new 1000 pseudo datasets in the ellipse area for the next iteration.
n=0
Ellipse_x=[]
Ellipse_y=[]
while n<1000:
    X =np.random.uniform(-a,a)
    Y =np.random.uniform(-b,b)
    d = (X / a) ** 2 + (Y / b) ** 2
    if d<1:
        Ellipse_x=np.append(Ellipse_x,X)
        Ellipse_y=np.append(Ellipse_y,Y)
        n=n+1
Ellipse_x=np.array([Ellipse_x])
Ellipse_y=np.array([Ellipse_y])

Ellipse = np.concatenate((Ellipse_x, Ellipse_y),axis=0)

Ellipse = numpy.dot(transform, Ellipse)
Ellipse[0, :] += mean_x
Ellipse[1, :] += mean_y

In [ ]:
Theil=np.loadtxt("pseudo-Theilindex-run2-1000.txt",delimiter=',')

from matplotlib import pyplot as plt

x = Theil[:,1]
y = Theil[:,2]
z = Theil[:,0]
color = [str(item/255.) for item in z]

plt.figure(figsize=(15,10))
plt.scatter(x, y, c=z, s=40, cmap=plt.cm.Blues)
plt.xlabel('generated Kaon mass [MeV/c^2]')
plt.ylabel('generated pion mass [MeV/c^2]')
cbar=plt.colorbar()
cbar.set_label('Theil index')
plot(ellipse[0],ellipse[1],'.r') #Potting the red ellipse.

savefig("run2-bin100x100.png")
